# Explore 3 simulated distorted targets in 12Mbps genome

In [4]:
%load_ext autoreload
%autoreload 2

import higlass as hg
import numpy as np

from utils import create_annotation_track, create_bed_vector_tileset, extract_annotations

genome_size = 1.2e7

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Tilesets

In [5]:
ts_signal_1 = hg.tilesets.bigwig('data/signal-target-1-distorted-peaks-12000000-1000000.bigWig')
ts_signal_2 = hg.tilesets.bigwig('data/signal-target-2-distorted-peaks-12000000-1000000.bigWig')
ts_signal_3 = hg.tilesets.bigwig('data/signal-target-3-distorted-peaks-12000000-1000000.bigWig')
ts_signal_baseline = hg.tilesets.bigwig('data/signal-baseline-distorted-peaks-12000000-1000000.bigWig')

ts_signal = [ts_signal_1, ts_signal_2, ts_signal_3]

ts_feature_weights = create_bed_vector_tileset(
    'data/simulated-features-3-targets-12000000-1000000.bed',
    'data/simulated-genome-chrom-sizes-12000000-1000000.tsv',
    uuid = 'feature_weights',
    aggregator = np.max,
    log_scale = True
)

annos_1, annos_2, annos_3, _, _, _, annos_all = extract_annotations(
    'data/simulated-features-3-targets-12000000-1000000.bed',
    {
        'BindingA': [0],
        'BindingB': [1],
        'BindingC': [2],
        'BindingAB': [0, 1, 3],
        'BindingAC': [0, 2, 4],
        'BindingBC': [1, 2, 5],
        'BindingABC': [0, 1, 2, 3, 4, 5, 6]
    },
    'data/simulated-genome-chrom-sizes-12000000-1000000.tsv',
)

### Tracks

In [11]:
tr_axis = hg.Track(track_type='top-axis', position='top', uid="axis")

tr_annos_1 = create_annotation_track('annos-1', annos_1)
tr_annos_2 = create_annotation_track('annos-2', annos_2)
tr_annos_3 = create_annotation_track('annos-3', annos_3)

tr_signal_baseline = hg.Track(
    'horizontal-bar',
    uid='signal-baseline',
    tileset=ts_signal_baseline,
    position='top',
    height=48,
    options={
        'name': 'Baseline reads',
        'labelColor': '#888888',
        'barFillColor': '#888888'
    }
)

tr_feature_weights = hg.Track(
    'horizontal-bar',
    uid='feature-weights',
    tileset=ts_feature_weights,
    position='top',
    height=16,
    options={
        "name": "Feature weights (log)",
        "colorRange": [
            "#f2f2f2", "#f2f2f2", "#f2f2f2", "#f2f2f2", "#dddddd", "#bbbbbb", "#888888"
        ],
        "labelColor": "#666666",
    }
)

tr_signal = []

for i in range(3):
    tr_signal.append(hg.Track(
        'horizontal-bar',
        uid=f"signal-{i + 1}",
        tileset=ts_signal[i],
        position='top',
        height=48,
        options={
            'name': f"Target {i + 1} signal",
            'labelColor': '#6fb2e4',
            'colorRange': ['#6fb2e4', '#0064a8'],
        }
    ))

overlays = [{
    "uid": "overlays",
    "includes": [
        "feature-weights",
        "annos-1",
        "annos-2",
        "annos-3",
        "signal-1",
        "signal-2",
        "signal-3",
        "signal-baseline",
    ],
    "options": {
        "extent": [
            [int(x[0]), int(x[1])] for x in annos_all
        ],
        "minWidth": 6,
        "fill": "#c17da5",
        "fillOpacity": 0.1,
        "stroke": "white",
        "strokeWidth": 1,
        "strokePos": ["left", "right"],
        "outline": "#c17da5",
        "outlineWidth": 2,
        "outlinePos": ["left", "right"]
    }
}]

widget, server, _ = hg.display(
    views=[
        hg.View(
            [
                tr_axis,
                tr_feature_weights,
                tr_annos_1,
                tr_annos_2,
                tr_annos_3,
                tr_signal[0],
                tr_signal[1],
                tr_signal[2],
                tr_signal_baseline,
            ],
            uid='v',
            initialXDomain=[0, genome_size],
            overlays=overlays
        )
    ],
    value_scale_syncs=[[
        tr_signal[0],
        tr_signal[1],
        tr_signal[2],
        tr_signal_baseline,
    ]],
    no_fuse=True
)

widget

env: HIGLASS_PYTHON_DEBUG=True


HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'v', 'tracks': {'top': [{'type': 'top-axis', 'uid…